In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import cv2
from tqdm import tqdm

In [2]:
IMAGES_PATH = 'C:/Users/luisf/Documents/Prog/Datasets/PhraseCutV0/images'
MASKS_PATH = 'C:/Users/luisf/Documents/Prog/Datasets/PhraseCutV0/masks'
ANNOTATIONS_PATH = 'C:/Users/luisf/Documents/Prog/Datasets/PhraseCutV0'

In [3]:
def preprocess_split(split_json):
    ind = 0

    images = []
    masks = []
    phrases = []

    for i in tqdm(split_json):
        img_path = 'PhraseCutDatasetD/data/VGPhraseCut_v0/images/' + str(i['image_id']) + '.jpg'
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        mask = np.zeros_like(img)
            
        for j in i['Polygons'][0]:
            cv2.fillPoly(mask, np.array([j], dtype=np.int32), (255, 255, 255))
            
        img = cv2.resize(img, (256, 256))
        mask = cv2.resize(mask, (256, 256))
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)

        cv2.imwrite(IMAGES_PATH + '/' + str(i['image_id']) + '.jpg', img)
        cv2.imwrite(MASKS_PATH + '/' + str(i['image_id'])+ '_' + str(ind) + '.jpg', mask)

        images.append(i['image_id'])
        phrases.append(i['phrase'])
        masks.append(str(i['image_id'])+ '_' + str(ind))

        ind += 1

    df = pd.DataFrame({'image': images, 'mask': masks, 'phrase': phrases})
    return df

In [4]:
train_path = 'PhraseCutDatasetD/data/VGPhraseCut_v0/refer_train.json'
val_path = 'PhraseCutDatasetD/data/VGPhraseCut_v0/refer_val.json'
test_path = 'PhraseCutDatasetD/data/VGPhraseCut_v0/refer_test.json'

In [5]:
with open(train_path) as f:
    train_json = json.load(f)

train_df = preprocess_split(train_json)
train_df.to_csv(ANNOTATIONS_PATH + '/train.csv', index=False)

del train_json
del train_df

with open(val_path) as f:
    val_json = json.load(f)

val_df = preprocess_split(val_json)
val_df.to_csv(ANNOTATIONS_PATH + '/val.csv', index=False)

del val_json
del val_df

with open(test_path) as f:
    test_json = json.load(f)

test_df = preprocess_split(test_json)
test_df.to_csv(ANNOTATIONS_PATH + '/test.csv', index=False)

del test_json
del test_df

100%|██████████| 14354/14354 [01:32<00:00, 154.94it/s]
